In [10]:
import itertools
import random

# --- 1. Random literal (with at most one 'not') ---
def random_literal(variables):
    var = random.choice(variables)
    return var if random.random() > 0.5 else f"not {var}"

# --- 2. Build a random formula recursively ---
def random_formula(variables, depth=2):
    if depth == 0:
        return random_literal(variables)
    left = random_formula(variables, depth - 1)
    right = random_formula(variables, depth - 1)
    op = random.choice(["and", "or", "->", "<->"])
    if op == "->":
        return f"(not {left} or {right})"  # p -> q ≡ ¬p ∨ q
    elif op == "<->":
        return f"(({left} and {right}) or (not {left} and not {right}))"
    else:
        return f"({left} {op} {right})"

# --- 3. Evaluate a formula for a given row of truth values ---
def eval_formula(formula, values):
    return eval(formula, {}, values)

# --- 4. Build truth table ---
def truth_table(variables, formulas):
    table = []
    for combo in itertools.product([False, True], repeat=len(variables)):
        vals = dict(zip(variables, combo))
        row = {v: vals[v] for v in variables}
        for label, f in formulas.items():
            row[label] = eval_formula(f, vals)
        table.append(row)
    return table

# --- 5. Generate a random argument ---
def random_argument(variables=["p", "q", "r"], num_premises=2):
    premises = [random_formula(variables) for _ in range(num_premises)]
    conclusion = random_formula(variables)
    return premises, conclusion

# --- 6. Check argument validity ---
def check_argument(premises, conclusion, variables):
    formulas = {f"P{i+1}": prem for i, prem in enumerate(premises)}
    formulas["Conclusion"] = conclusion
    table = truth_table(variables, formulas)
    valid = True
    counterexamples = []
    for row in table:
        if all(row[f"P{i+1}"] for i in range(len(premises))) and not row["Conclusion"]:
            valid = False
            counterexamples.append(row)
    return valid, counterexamples

# --- 7. Pretty-print formulas with logical symbols ---
def pretty(formula):
    # Replace carefully: longer substrings first
    formula = formula.replace("<->", "↔")
    formula = formula.replace("->", "→")
    formula = formula.replace(" and ", " ∧ ")
    formula = formula.replace(" or ", " ∨ ")
    formula = formula.replace("not ", "¬")
    return formula


In [11]:

# --- Demo run ---
premises, conclusion = random_argument()
print("🧠 Randomly generated argument:")
for i, p in enumerate(premises, 1):
    print(f"  Premise {i}: {pretty(p)}")
print(f"  Conclusion: {pretty(conclusion)}")

valid, counterexamples = check_argument(premises, conclusion, ["p", "q", "r"])

print("\n📊 Result:")
if valid:
    print("✅ The argument is VALID (no counterexamples).")
else:
    print("❌ The argument is INVALID. Counterexamples:")
    for c in counterexamples:
        print("   " + ", ".join(f"{k}={v}" for k, v in c.items()))


🧠 Randomly generated argument:
  Premise 1: (¬(r ∨ p) ∨ (¬p ∨ ¬r))
  Premise 2: (((¬p ∧ ¬r) ∧ (r ∨ ¬q)) ∨ (¬(¬p ∧ ¬r) ∧ ¬(r ∨ ¬q)))
  Conclusion: ((¬r ∨ q) ∨ (q ∨ ¬q))

📊 Result:
✅ The argument is VALID (no counterexamples).
